In [16]:
import sqlite3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import classification_report
from sklearn.utils import shuffle

In [19]:
db_path = "../../db.sqlite3" 
conn = sqlite3.connect(db_path)

In [20]:
query = """
SELECT 
    orders.client_id,
    menu.name AS pizza_name,
    orders.order_date
FROM 
    orders
JOIN 
    menu
ON 
    orders.pizza_id = menu.pizza_id
"""
data = pd.read_sql_query(query, conn)

# Ajout d'une colonne pizza précédente
data = data.sort_values(by=['client_id', 'order_date'])
data['prev_pizza_1'] = data.groupby('client_id')['pizza_name'].shift(1)
data['prev_pizza_2'] = data.groupby('client_id')['pizza_name'].shift(2)
data['prev_pizza_3'] = data.groupby('client_id')['pizza_name'].shift(3)

#Supprimer la première commande sans antécédant
data = data.dropna(subset=['prev_pizza_1'])

# Récupérer tous les clients et split par client
client_ids = data['client_id'].unique()
train_clients, test_clients = train_test_split(client_ids, test_size=0.2, random_state=42)

# Séparer les commandes associées aux clients
train_data = data[data['client_id'].isin(train_clients)]
test_data = data[data['client_id'].isin(test_clients)]


encoder = OneHotEncoder(sparse_output=False)
X_train = encoder.fit_transform(train_data[['prev_pizza_1', 'prev_pizza_2', 'prev_pizza_3']].fillna('None'))
X_test = encoder.transform(test_data[['prev_pizza_1', 'prev_pizza_2', 'prev_pizza_3']].fillna('None'))



In [21]:
# Création du dataset d'entraînement
y_train = train_data['pizza_name']
y_test = test_data['pizza_name']

# Modèle de régression logistique
model = LogisticRegression(max_iter=1000, multi_class='multinomial')
model.fit(X_train, y_train)

c:\Users\barce\Desktop\Cours\3A\Hackathon\Opti-Loyalty\env\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [33]:
# Prédictions
y_pred_proba = model.predict_proba(X_test)
y_pred_top3 = np.argsort(y_pred_proba, axis=1)[:, -3:][:, ::-1]  # Indices des 3 meilleures prédictions

# Traduire les indices en noms de pizzas
top3_pizzas = encoder.categories_[0]
y_pred_top3_names = [[top3_pizzas[i] for i in indices] for indices in y_pred_top3]

# Fonction pour calculer l'accuracy pour Top k
def calculate_top_k_accuracy(y_test, y_pred_top3, k):
    correct_predictions = 0
    total_predictions = len(y_test)
    for true_pizza, predicted_pizzas in zip(y_test, y_pred_top3):
        if true_pizza in predicted_pizzas[:k]:
            correct_predictions += 1
    return correct_predictions / total_predictions

# Calcul des métriques
top1_accuracy = calculate_top_k_accuracy(y_test.tolist(), y_pred_top3_names, 1)
top2_accuracy = calculate_top_k_accuracy(y_test.tolist(), y_pred_top3_names, 2)
top3_accuracy = calculate_top_k_accuracy(y_test.tolist(), y_pred_top3_names, 3)

# Affichage des métriques
print(f"Accuracy Top 1 : {top1_accuracy:.2f}")
print(f"Accuracy Top 2 : {top2_accuracy:.2f}")
print(f"Accuracy Top 3 : {top3_accuracy:.2f}")

# Affichage des recommandations pour un client spécifique
client_index = 1
test_client = test_clients[client_index]
client_data = test_data[test_data['client_id'] == test_client]
print(f"\nRecommandations pour le client {test_client} :")
relative_index = 0
for _, row in client_data.iterrows():
    print(f"Pizzas précédentes : {row['prev_pizza_1']}, {row['prev_pizza_2']}, {row['prev_pizza_3']} | Top 3 recommandées : {y_pred_top3_names[relative_index]}")
    relative_index += 1

Accuracy Top 1 : 0.19
Accuracy Top 2 : 0.30
Accuracy Top 3 : 0.34

Recommandations pour le client 2945 :
Pizzas précédentes : The Four Cheese Pizza, nan, nan | Top 3 recommandées : ['The Hawaiian Pizza', 'The Thai Chicken Pizza', 'The Pepperoni Pizza']
Pizzas précédentes : The Greek Pizza, The Four Cheese Pizza, nan | Top 3 recommandées : ['The Hawaiian Pizza', 'The Thai Chicken Pizza', 'The Four Cheese Pizza']
Pizzas précédentes : The Four Cheese Pizza, The Greek Pizza, The Four Cheese Pizza | Top 3 recommandées : ['The Hawaiian Pizza', 'The Barbecue Chicken Pizza', 'The Thai Chicken Pizza']
Pizzas précédentes : The Napolitana Pizza, The Four Cheese Pizza, The Greek Pizza | Top 3 recommandées : ['The Hawaiian Pizza', 'The Sicilian Pizza', 'The Barbecue Chicken Pizza']
Pizzas précédentes : The Greek Pizza, The Napolitana Pizza, The Four Cheese Pizza | Top 3 recommandées : ['The Pepperoni, Mushroom, and Peppers Pizza', 'The Classic Deluxe Pizza', 'The Barbecue Chicken Pizza']
Pizzas pré